<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

In [1]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = "../logs/"
MASK_RCNN_MODEL_PATH = "../models/mask_rcnn_numberplate_0700.h5"
OPTIONS_MODEL_PATH =  "../models/numberplate_options_2019_2_15.h5"

# If you use gpu version tensorflow please change model to gpu version named like *-gpu.h5
OCR_NP_UKR_TEXT =  "../models/anpr_ocr_ua_1_2_11-cpu.h5"
OCR_NP_EU_TEXT =  "../models/anpr_ocr_eu_2-cpu.h5"

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync
from NomeroffNet.Base import convert_keras_to_freeze_pb

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load(OPTIONS_MODEL_PATH)

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2014_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": OCR_NP_UKR_TEXT
    },
    "eu": {
        "for_regions": ["eu", "eu_ua_1995"],
        "model_path": OCR_NP_EU_TEXT
    }
})

Using TensorFlow backend.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


### Convert keras options .h5 model to .pb graph

In [3]:
OPTION_DETECTOR_FROZEN_MODEL_PATH = "../models/numberplate_options_2019_2_15.pb"

convert_keras_to_freeze_pb(optionsDetector.MODEL, OPTION_DETECTOR_FROZEN_MODEL_PATH)

OUTPUT: REGION/Softmax,STATE/Softmax
INPUT: input_2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 295, 3)   0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    input_2[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 9216)         0           vgg16[1][0]                      
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 9216)         0           vgg16[1][0]                      
_________________________________________________________

### Convert keras OCR  .h5 model to .pb graph

In [4]:
from NomeroffNet.Base import OCR

OCR_NP_UKR_FROZEN_MODEL_PATH = "../models/anpr_ocr_ua_1_2_11-cpu.pb"
model = textDetector.get_module("eu_ua_2014_2015").MODEL
convert_keras_to_freeze_pb(model, OCR_NP_UKR_FROZEN_MODEL_PATH)

OUTPUT: softmax/truediv
INPUT: the_input
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
____________________________________________________________________

In [5]:
OCR_NP_EU_FROZEN_MODEL_PATH = "../models/anpr_ocr_eu_2-cpu.pb"
model = textDetector.get_module("eu").MODEL
convert_keras_to_freeze_pb(model, OCR_NP_EU_FROZEN_MODEL_PATH)

OUTPUT: softmax_1/truediv
INPUT: the_input_1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
________________________________________________________________

### Convert MaskRCNN .h5 model  to frozen .pb graph

In [6]:
MASK_RCNN_FROZEN_MODEL_PATH = "../models/mask_rcnn_numberplate_0700.pb"
model = nnet.getKerasModel()

convert_keras_to_freeze_pb(model,MASK_RCNN_FROZEN_MODEL_PATH)

OUTPUT: mrcnn_detection/Reshape_1,mrcnn_class/Reshape_1,mrcnn_bbox/Reshape,mrcnn_mask/Reshape_1,ROI/packed_2,rpn_class/concat,rpn_bbox/concat
INPUT: input_image,input_image_meta,input_anchors
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchN

INFO:tensorflow:Restoring parameters from ./saved_ckpt-0
INFO:tensorflow:Froze 690 variables.
INFO:tensorflow:Converted 690 variables to const ops.
